# Tavily research endpoint with streaming

Next, let's explore streaming. Streaming allows you to view real-time progress updates for your research job, including events such as tool calls, tool responses, and content generation.

Example streaming response is shown below for reference.


In [ ]:
%pip install -qU tavily-python pydantic

In [ ]:
import getpass
import os
import json
from IPython.display import display, Markdown
from tavily import TavilyClient

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [ ]:
input="What are the latest developments in artificial intelligence?"
model = "mini"  # model = "mini" | "pro" | "auto"

In [ ]:
print(f"\n{'='*80}")
print(f"Starting Research: {input}")
print(f"Model: {model}")
print(f"{'='*80}\n")

response = tavily_client.research(input=input, model=model, stream=True)
full_report = ""

current_step = None
event_type = None

for chunk in response:
    # Decode bytes to string
    line = chunk.decode("utf-8").strip() if isinstance(chunk, bytes) else str(chunk).strip()
    
    if not line:
        continue
    
    # Parse SSE format
    if line.startswith("event:"):
        event_type = line.split("event:", 1)[1].strip()
    elif line.startswith("data:"):
        data_str = line.split("data:", 1)[1].strip()
        data = json.loads(data_str)
        
        if event_type == "chat.completion.chunk":
            delta = data.get("choices", [{}])[0].get("delta", {})
            
            # Handle content chunks (final report)
            if "content" in delta:
                content = delta["content"]
                full_report += content
                print(content, end="", flush=True)
            
            # Handle step details (research progress)
            elif "step_details" in delta:
                step = delta["step_details"]
                step_type = step.get("type", "")
                
                if step_type == "research_plan":
                    print(f"\n📋 Research Plan:")
                    print(f"   {step.get('step', 'N/A')}")
                
                elif step_type == "research":
                    research_input = step.get("step", "")
                    if current_step != research_input:
                        current_step = research_input
                        print(f"\n\n🔍 Researching: {research_input}")
                
                elif step_type == "think":
                    print(f"\n💭 Thinking: {step.get('step', 'N/A')}")
            
            # Handle tool calls
            elif "tool_calls" in delta:
                tool_calls_data = delta.get("tool_calls", {})
                if isinstance(tool_calls_data, dict):
                    call_type = tool_calls_data.get("type", "")
                    
                    # Get the actual array of tool calls/responses
                    items = tool_calls_data.get(call_type, [])
                    
                    for item in items:
                        if isinstance(item, dict):
                            arguments = item.get("arguments", "")
                            if arguments and call_type:
                                print(f"\n🔧 Tool: {arguments}")

print(f"\n\n{'='*80}")
print("Research Complete!")
print(f"{'='*80}\n")

In [ ]:
display(Markdown(full_report))